Here we extract the data from tensorboard logs to plot the loss in our own figure.

This is for the DRAKKAR poster.

In [1]:
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorboard.util import tensor_util 

In [2]:
def get_loss_dataframe(directory):
    ea = EventAccumulator(directory, size_guidance={"tensors": 0})
    ea.Reload()  # Load the events from the event files
    te = ea.Tensors('epoch_loss')
    
    df = pd.DataFrame({
        "step": [e.step for e in te],
        "wall_time": [e.wall_time for e in te],
        "train_epoch_loss": [float(tensor_util.make_ndarray(e.tensor_proto).item()) for e in te],
    }).sort_values("step")
    
    return df

Get the training loss

In [3]:
directory = 'logs/fit/20260122-152652/train/'
train_df = get_loss_dataframe(directory)
train_df

2026-01-23 10:59:39.425635: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-23 10:59:39.472593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-23 10:59:42.303453: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


,step,wall_time,train_epoch_loss
0,0,1.769096e+09,7.517107
1,1,1.769096e+09,3.342733
2,2,1.769096e+09,1.011117
3,3,1.769096e+09,0.528886
4,4,1.769096e+09,0.430608
...,...,...,...
95,95,1.769098e+09,0.169860
96,96,1.769098e+09,0.169679
97,97,1.769098e+09,0.170415
98,98,1.769098e+09,0.169235


Get the validation loss

In [4]:
directory = 'logs/fit/20260122-152652/validation/'
val_df = get_loss_dataframe(directory)
val_df

,step,wall_time,train_epoch_loss
0,0,1.769096e+09,8.707306
1,1,1.769096e+09,4.038942
2,2,1.769096e+09,1.601130
3,3,1.769096e+09,0.910873
4,4,1.769096e+09,0.775515
...,...,...,...
95,95,1.769098e+09,0.265880
96,96,1.769098e+09,0.269670
97,97,1.769098e+09,0.266094
98,98,1.769098e+09,0.270200


Merge and save dataframe

In [5]:
df = pd.merge(train_df, val_df, on='step', suffixes=('_train', '_val'))
df

,step,wall_time_train,train_epoch_loss_train,wall_time_val,train_epoch_loss_val
0,0,1.769096e+09,7.517107,1.769096e+09,8.707306
1,1,1.769096e+09,3.342733,1.769096e+09,4.038942
2,2,1.769096e+09,1.011117,1.769096e+09,1.601130
3,3,1.769096e+09,0.528886,1.769096e+09,0.910873
4,4,1.769096e+09,0.430608,1.769096e+09,0.775515
...,...,...,...,...,...
95,95,1.769098e+09,0.169860,1.769098e+09,0.265880
96,96,1.769098e+09,0.169679,1.769098e+09,0.269670
97,97,1.769098e+09,0.170415,1.769098e+09,0.266094
98,98,1.769098e+09,0.169235,1.769098e+09,0.270200


In [6]:
df.to_pickle('loss_data/cnn_loss_20260122-152652_SO_JET.pkl')